# Items demand utility

Import all the needed modules

In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np

Import from the statistics file the columns needed

In [2]:
# From Warehouse / Item Statistics / Item Statistics
excel_file = 'R12.xlsx'
df = pd.read_excel(excel_file, usecols = ['Item', 'Year month',
                                          'Sold qty', 'Used qty',
                                          'Not rep usage'])

Import from the master file the columns needed

In [3]:
# From Data / Item Master / Item Master
master_file = 'Item master.xlsx'
df_master = pd.read_excel(master_file, usecols = ['Item no', 'Item name',
                                            'Product group descr'])
df_master['Item no'] = df_master['Item no'].astype(str)

Import from Open PO

In [4]:
# From Supply / Open PO & DO
excel_file = 'Open PO.xlsx'
df_po = pd.read_excel(excel_file, usecols = ['Order no', 'Item no',
                                             'Ordered qty', 'Shipping date'])
df_po = df_po[1:].reset_index(drop=True)

cols = ['Order no', 'Item no']
for col in cols:
    df_po[col] = df_po[col].astype(str)
    
df_po['Ordered qty'] = df_po['Ordered qty'].astype(float)

Import supply data for suppliers

In [5]:
# From Supply / PO & DO Statistics / Purchase statistics lines
excel_file = 'Supply.xlsx'
df_supply = pd.read_excel(excel_file, usecols = ['Item no', 'Supplier'])
df_supply.drop_duplicates(inplace=True)
df_supply['Item no'] = df_supply['Item no'].astype(str)

Import Item Balance

In [6]:
# From Warehouse / Availability / Item Balance
excel_file = 'Item balance.xlsx'
df_balance = pd.read_excel(excel_file, usecols =
                           ['Item no', 'Safety stock','Reserved not alloc', 'On hand bal'])
df_balance['Item no'] = df_balance['Item no'].astype(str)

Get the first word of the item column using str.split and str.get function

In [7]:
df['Item no'] = df['Item'].str.split().str.get(0)
df['Item no'] = df['Item no'].astype(str)

Remove possible whitespaces in the figures

In [8]:
columns = ['Sold qty', 'Used qty', 'Not rep usage']

for column in columns:
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace(' ', '')
    df[column] = df[column].astype(float)

Get the demand

In [9]:
df['Demand'] = df['Sold qty'] + df['Used qty'] + np.where(df['Not rep usage'] > 0, df['Not rep usage'], 0)

Get the pivot table of the demand and year month and replace NaN with zeros

In [10]:
pivot_df = pd.pivot_table(df, values = 'Demand', index = 'Item no' , columns = 'Year month', aggfunc=np.sum)
pivot_df = pivot_df.fillna(0)
pivot_df = pd.DataFrame(pivot_df.to_records())
pivot_df.head()

,Item no,1909,1910,1911,1912,2001,2002,2003,2004,2005,2006,2007,2008
0,0645000,0.0,99.0,297.0,99.0,198.0,33.0,99.0,99.0,198.0,99.0,33.0,33.0
1,0645001,39.0,78.0,39.0,39.0,156.0,78.0,156.0,39.0,78.0,39.0,117.0,78.0
2,0645002,135.0,90.0,45.0,45.0,0.0,45.0,180.0,0.0,90.0,45.0,135.0,45.0
3,0645003,153.0,459.0,102.0,0.0,51.0,102.0,0.0,51.0,204.0,102.0,153.0,102.0
4,0645004,232.0,232.0,58.0,58.0,464.0,232.0,102.0,0.0,102.0,102.0,204.0,102.0


Total field for pivot_df

In [11]:
col_len = len(pivot_df.columns)
pivot_df['Total'] = pivot_df.iloc[:, 1:col_len].sum(axis=1, numeric_only=True)
pivot_df['Mean'] = pivot_df.iloc[:, 1:col_len].mean(axis=1, numeric_only=True)
pivot_df['StD'] = pivot_df.iloc[:, 1:col_len].std(axis=1, numeric_only=True) 
pivot_df['CV'] = pivot_df['StD'] / pivot_df['Mean']
pivot_df['Min'] = pivot_df.iloc[:, 1:col_len].min(axis=1, numeric_only=True)
pivot_df['Q25'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.25, axis=1, numeric_only=True).round(0)
pivot_df['Median'] = pivot_df.iloc[:, 1:col_len].median(axis=1, numeric_only=True)
pivot_df['Q67'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.67, axis=1, numeric_only=True).round(0)
pivot_df['Q75'] = pivot_df.iloc[:, 1:col_len].quantile(q=0.75, axis=1, numeric_only=True).round(0)
pivot_df['Max'] = pivot_df.iloc[:, 1:col_len].max(axis=1, numeric_only=True)
pivot_df['Item no'] = pivot_df['Item no'].astype(str)
pivot_df.head()

,Item no,1909,1910,1911,1912,2001,2002,2003,2004,2005,...,Total,Mean,StD,CV,Min,Q25,Median,Q67,Q75,Max
0,0645000,0.0,99.0,297.0,99.0,198.0,33.0,99.0,99.0,198.0,...,1287.0,107.250000,85.736515,0.799408,0.0,33.0,99.0,99.0,124.0,297.0
1,0645001,39.0,78.0,39.0,39.0,156.0,78.0,156.0,39.0,78.0,...,936.0,78.000000,43.997934,0.564076,39.0,39.0,78.0,78.0,88.0,156.0
2,0645002,135.0,90.0,45.0,45.0,0.0,45.0,180.0,0.0,90.0,...,855.0,71.250000,55.805058,0.783229,0.0,45.0,45.0,90.0,101.0,180.0
3,0645003,153.0,459.0,102.0,0.0,51.0,102.0,0.0,51.0,204.0,...,1479.0,123.250000,121.971029,0.989623,0.0,51.0,102.0,121.0,153.0,459.0
4,0645004,232.0,232.0,58.0,58.0,464.0,232.0,102.0,0.0,102.0,...,1888.0,157.333333,124.200229,0.789408,0.0,91.0,102.0,214.0,232.0,464.0


Get the 'Item name' for each item. Merge it with the master_df

In [12]:
df_master = pd.merge(df_master, df_supply, on='Item no') # Suppliers

df_master_balance = pd.merge(df_master, df_balance, on='Item no')
df_master_balance['Item no'] = df_master_balance['Item no'].astype(str)

demand_df = pd.merge(df_master, pivot_df, on='Item no')
demand_df = demand_df.fillna(0)

demand_df_balance = pd.merge(df_master_balance, pivot_df, on='Item no')
demand_df_balance = demand_df_balance.fillna(0)

New columns

In [13]:
df_po_match = df_po.loc[df_po['Item no'].isin(df_master_balance['Item no']), :]
data = []
for row in range(len(df_po_match['Item no'])):
    item = df_po_match['Item no'].iloc[row]
    df_filt = df_po_match.loc[df_po_match['Item no'] == item, :]
    total = df_filt['Ordered qty'].sum()
    data.append([item, total])

# Create dataframe
cols = ['Item no', 'Pending PO Qty']
df_pending = pd.DataFrame(data, columns = cols)

# Merge
df_merge = pd.merge(demand_df_balance, df_pending, on='Item no', how='left')
df_merge['Pending PO Qty'] = df_merge['Pending PO Qty'].fillna(0)

In [14]:
df_merge['Months availability'] = (df_merge['On hand bal'].values - df_merge['Reserved not alloc'].values)/df_merge['Mean'].values

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [15]:
excel_file = 'Lead time.xlsx'
df_lead = pd.read_excel(excel_file)
df_merge = pd.merge(df_merge, df_lead, on='Supplier', how='left')

,Item no,Item name,Product group descr,Supplier,Safety stock,Reserved not alloc,On hand bal,1909,1910,1911,...,CV,Min,Q25,Median,Q67,Q75,Max,Pending PO Qty,Months availability,Lead time
0,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,2.891995,0.0,0.0,0.0,0.0,0.0,5.0,0.0,18.0,30
1,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,2.891995,0.0,0.0,0.0,0.0,0.0,5.0,0.0,18.0,30
2,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,2.891995,0.0,0.0,0.0,0.0,0.0,5.0,0.0,18.0,30
3,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,2.891995,0.0,0.0,0.0,0.0,0.0,5.0,0.0,18.0,30
4,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,2.891995,0.0,0.0,0.0,0.0,0.0,5.0,0.0,18.0,30


In [16]:
df_merge['Sobrestock'] = (df_merge['Months availability']-df_merge['Lead time'])/df_merge['Lead time']

In [17]:
df_merge['SS'] = df_merge['Lead time']*df_merge['Mean']
df_merge['RoP'] = 2*df_merge['SS']
df_merge['Q'] = df_merge['Mean'] + df_merge['SS']*df_merge['Lead time']

In [18]:
excel_file = 'MMS003_prices.xlsx'
df_price = pd.read_excel(excel_file, usecols=['Item no', 'Prod cost'])

In [19]:
df_merge = pd.merge(df_merge, df_price, on='Item no', how='left')
df_merge.head()

,Item no,Item name,Product group descr,Supplier,Safety stock,Reserved not alloc,On hand bal,1909,1910,1911,...,Q75,Max,Pending PO Qty,Months availability,Lead time,Sobrestock,SS,RoP,Q,Prod cost
0,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,0.0,5.0,0.0,18.0,30,-0.4,15.0,30.0,450.5,74.931
1,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,0.0,5.0,0.0,18.0,30,-0.4,15.0,30.0,450.5,74.931
2,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,0.0,5.0,0.0,18.0,30,-0.4,15.0,30.0,450.5,74.931
3,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,0.0,5.0,0.0,18.0,30,-0.4,15.0,30.0,450.5,74.931
4,1330,TFSR 125 M Black,203 Roof fans,S010 Systemair Sverige AB,5.0,0.0,9.0,0.0,1.0,0.0,...,0.0,5.0,0.0,18.0,30,-0.4,15.0,30.0,450.5,74.931


In [21]:
df_merge = df_merge.drop_duplicates(subset = 'Item no', keep = 'last')

Output the demand to Excel

In [22]:
name = 'Items Demand.xlsx'
writer = pd.ExcelWriter(name)
demand_df.to_excel(writer, index = False)
writer.save()
print('Done bro!')

name = 'Items Demand Balance.xlsx'
writer = pd.ExcelWriter(name)
df_merge.to_excel(writer, index = False)
writer.save()
print('Done bro!')

Done bro!
Done bro!
